### Simple convolutional neural network 

Data by @RDizzl3 https://www.kaggle.com/c/happy-whale-and-dolphin/discussion/304686

WIP 

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
import cv2
import glob
import matplotlib.pyplot as plt

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
df = pd.read_csv("../input/happy-whale-and-dolphin/train.csv")

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
# Get the train image file names
# Use the * as a wild card this will tell glob get us all images in this directory
image_path = '/kaggle/input/jpeg-happywhale-128x128/train_images-128-128/train_images-128-128/*'
train_filenames = glob.glob(image_path)

In [ ]:
# Get a single image path
image_path = train_filenames[0]
img = cv2.imread(image_path)

# Check the shape - should be (128, 128, 3)
img.shape

In [ ]:
# Plot the image
plt.imshow(img)

In [ ]:
df.head()

In [ ]:

train_df = df[['image','individual_id']]

In [ ]:
train_df.head()

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,Conv2D,MaxPooling2D,Flatten,Reshape,Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
callbacks = [
            EarlyStopping(patience = 3,monitor="accuracy",mode="max"),    
            ]

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255, 
                                   shear_range=0.2,
                                   zoom_range=0.2, 
                                   horizontal_flip=True,
                                  width_shift_range=0.1,
                                  height_shift_range=0.1,
                                   validation_split=0.2
                                   )

In [ ]:
validation_datagen = ImageDataGenerator(rescale=1./255,validation_split=0.2)

In [ ]:
train_dir = '/kaggle/input/jpeg-happywhale-128x128/train_images-128-128/train_images-128-128/'

In [ ]:
df.head()

In [ ]:
train_df.head()

In [ ]:
train_dir

In [ ]:
individuals = list(df['individual_id'].unique())

In [ ]:
training_set = train_datagen.flow_from_dataframe(
                                                train_df,
                                                train_dir,
                                                seed=101,                                                 
                                                target_size=(64, 64),
                                                labels = individuals,
                                                batch_size=32,
                                                x_col='image',
                                                y_col='individual_id',
                                                class_mode='categorical',
                                                subset = "training"
                                                )

In [ ]:
validation_set = validation_datagen.flow_from_dataframe(
                                                train_df,
                                                train_dir,
                                                seed=101,                                                 
                                                target_size=(64, 64),
                                                labels = individuals,
                                                batch_size=32,
                                                x_col='image',
                                                y_col='individual_id',
                                                class_mode='categorical',
                                                subset = "validation"
                                                )

In [ ]:
model = Sequential([
    Conv2D(filters=32,kernel_size=(3,3), input_shape = (64, 64, 3), activation='relu'),
    MaxPooling2D(pool_size=(2,2)),

    Conv2D(filters=32,kernel_size=(3,3), activation='relu'),
    MaxPooling2D(pool_size=(2,2)),
    Dropout(0.25),

    Conv2D(filters=64,kernel_size=(3,3), activation='relu'),
    MaxPooling2D(pool_size=(2,2)),
    Dropout(0.25),
    Dense(128, activation='relu'),
    Flatten(),
    Dense(15587, activation='softmax')
])

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
 epochs=100 

In [ ]:
history = model.fit(
          training_set,
          epochs=epochs, 
        validation_data=validation_set,

          callbacks=callbacks)